# 项目：整理Netflix电影演员评分数据

## 分析目标

此数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

本实战项目的目的在于练习整理数据，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了截止至2022年7月美国地区可观看的所有Netflix电视剧及电影数据。数据集包含两个数据表：`titles.csv`和`credits.csv`。

`titles.csv`包含电影及电视剧相关信息，包括影视作品ID、标题、类型、描述、流派、IMDB（一个国外的在线评分网站）评分，等等。`credits.csv`包含超过7万名出现在Netflix影视作品的导演及演员信息，包括名字、影视作品ID、人物名、演职员类型（导演/演员）等。

`titles.csv`每列的含义如下：
- id：影视作品ID。
- title：影视作品标题。
- show_type：作品类型，电视节目或电影。
- description：简短描述。
- release_year：发布年份。
- age_certification：适龄认证。
- runtime：每集电视剧或电影的长度。
- genres：流派类型列表。
- production_countries：出品国家列表。
- seasons：如果是电视剧，则是季数。
- imdb_id：IMDB的ID。
- imdb_score：IMDB的评分。
- imdb_votes：IMDB的投票数。
- tmdb_popularity：TMDB的流行度。
- tmdb_score：TMDB的评分。

`credits.csv`每列的含义如下：
- person_ID：演职员ID。
- id：参与的影视作品ID。
- name：姓名。
- character_name：角色姓名。
- role：演职员类型，演员或导演。

In [1]:
import pandas as pd

### 读取数据

In [2]:
original_data_1 = pd.read_csv('titles.csv')
original_data_2 = pd.read_csv('credits.csv')

In [3]:
original_data_1.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


### 评估及清洗数据

In [4]:
original_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5850 non-null   object 
 1   title                 5849 non-null   object 
 2   type                  5850 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5850 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5850 non-null   int64  
 7   genres                5850 non-null   object 
 8   production_countries  5850 non-null   object 
 9   seasons               2106 non-null   float64
 10  imdb_id               5447 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score            5539 non-null   float64
dtypes: float64(5), int64(

In [5]:
original_data_2.head()

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


先对两个表格进行数据清洗，合格的数据在结构上需要满足每列为一个变量、每行为一个观察值，每个单元格为一个值；在内容上需要满足非空数据、非不一致数据，非重复数据。

In [6]:
original_data_1['id'].value_counts()

id
ts300399    1
tm912400    1
ts89361     1
tm494531    1
tm857921    1
           ..
tm217228    1
tm420745    1
ts57082     1
tm363248    1
ts271048    1
Name: count, Length: 5850, dtype: int64

检查original_data_1中id是否有重复现象，根据上表可以看出id无重复

观察表一，发现表一的 genres 列和 production_countries列在结构上不符合每列为一个变量的要求，由于 genres 数据与项目目标相关需要对其进行拆分；production_countries 数据与项目目标无关，可以先不予处理。

In [7]:
cleaned_data_1 = original_data_1.copy()
cleaned_data_2 = original_data_2.copy()
cleaned_data_1['genres'][0]

"['documentation']"

先观察 genres 列的其中一个值，发现其为字符串类型。为了拆分，需要将其变为列表类型。

In [8]:
cleaned_data_1['genres'] = cleaned_data_1['genres'].apply(eval)
cleaned_data_1['genres'][0]

['documentation']

In [9]:
cleaned_data_1 = cleaned_data_1.explode('genres')
cleaned_data_1['genres']

0       documentation
1               drama
1               crime
2               drama
2              action
            ...      
5847           comedy
5848              NaN
5849           family
5849        animation
5849           comedy
Name: genres, Length: 15147, dtype: object

对 genres 的拆分完成，保证了 genres列内为一个变量的要求。在内容上， genres可能存在一种类型多种表示方式的不一致数据，需要进行检查。

In [10]:
cleaned_data_1['genres'].value_counts()

genres
drama            2968
comedy           2325
thriller         1228
action           1157
romance           971
documentation     952
crime             936
animation         705
family            682
fantasy           630
scifi             589
european          443
horror            378
music             262
history           254
reality           234
sport             170
war               163
western            41
Name: count, dtype: int64

数据无异常

In [11]:
cleaned_data_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15147 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    15147 non-null  object 
 1   title                 15146 non-null  object 
 2   type                  15147 non-null  object 
 3   description           15125 non-null  object 
 4   release_year          15147 non-null  int64  
 5   age_certification     9298 non-null   object 
 6   runtime               15147 non-null  int64  
 7   genres                15088 non-null  object 
 8   production_countries  15147 non-null  object 
 9   seasons               5923 non-null   float64
 10  imdb_id               14525 non-null  object 
 11  imdb_score            14399 non-null  float64
 12  imdb_votes            14375 non-null  float64
 13  tmdb_popularity       14995 non-null  float64
 14  tmdb_score            14614 non-null  float64
dtypes: float64(5), int64(2), 

In [12]:
cleaned_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


由于需要整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。所以只需保证
id、genres、imdb_score、person_ID、name和role数据的无误即可。因此需要删除这些列为空的观察值以免对项目结果造成误差。

In [13]:
cleaned_data_1 = cleaned_data_1.reset_index().drop('index',axis=1)

In [14]:
cleaned_data_1 = cleaned_data_1.dropna(subset=['genres'])

删除 genres列为空的观察值

In [15]:
cleaned_data_1 = cleaned_data_1.dropna(subset=['imdb_score'])

删除 imdb_score列为空的观察值

In [16]:
cleaned_data_2 = cleaned_data_2[cleaned_data_2['role'] == 'ACTOR']
(cleaned_data_2['role'] !=  'ACTOR').sum()

0

由于项目需要挖掘演员而非导演，故删除 role 为导演的观察值

In [17]:
cleaned_data_1.describe()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,14393.000000,14393.000000,5655.000000,14393.000000,1.437100e+04,14268.000000,13975.000000
mean,2015.994164,78.326270,2.424934,6.534802,2.836162e+04,30.008057,6.885143
std,7.156617,40.555015,2.867856,1.157513,1.041849e+05,99.359539,1.112448
min,1954.000000,0.000000,1.000000,1.500000,5.000000e+00,0.600000,1.000000
25%,2015.000000,44.000000,1.000000,5.800000,7.080000e+02,3.911750,6.200000
50%,2018.000000,87.000000,1.000000,6.600000,3.166000e+03,9.666000,6.985000
75%,2020.000000,106.000000,3.000000,7.400000,1.432100e+04,22.766000,7.600000
max,2022.000000,225.000000,42.000000,9.500000,2.294231e+06,2274.044000,10.000000


In [18]:
cleaned_data_2.describe()

,person_id
count,7.325100e+04
mean,5.342692e+05
std,6.472396e+05
min,7.000000e+00
25%,4.400200e+04
50%,1.973050e+05
75%,9.006610e+05
max,2.462818e+06


通过describe方法可以发现cleaned_data_和cleaned_data_2 均无异常值。

### 数据整理

In [19]:
actor_data = pd.merge(cleaned_data_1,cleaned_data_2, on = 'id',how ='inner')
actor_data

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,3748,Robert De Niro,Travis Bickle,ACTOR
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,14658,Jodie Foster,Iris Steensma,ACTOR
2,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,7064,Albert Brooks,Tom,ACTOR
3,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,3739,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,48933,Cybill Shepherd,Betsy,ACTOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205436,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,comedy,['CO'],NaN,tt14585902,3.8,68.0,26.005,6.300,368473,Aída Morales,Maritza,ACTOR
205437,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,comedy,['CO'],NaN,tt14585902,3.8,68.0,26.005,6.300,736339,Adelaida Buscato,María Paz,ACTOR
205438,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,comedy,['CO'],NaN,tt14585902,3.8,68.0,26.005,6.300,399499,Luz Stella Luengas,Karen Bayona,ACTOR
205439,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,comedy,['CO'],NaN,tt14585902,3.8,68.0,26.005,6.300,373198,Inés Prieto,Fanny,ACTOR


通过 id 的连接合并列表

In [20]:
actor_data = actor_data[['id','genres','imdb_score','person_id','name']]
actor_data

,id,genres,imdb_score,person_id,name
0,tm84618,drama,8.2,3748,Robert De Niro
1,tm84618,drama,8.2,14658,Jodie Foster
2,tm84618,drama,8.2,7064,Albert Brooks
3,tm84618,drama,8.2,3739,Harvey Keitel
4,tm84618,drama,8.2,48933,Cybill Shepherd
...,...,...,...,...,...
205436,tm1059008,comedy,3.8,368473,Aída Morales
205437,tm1059008,comedy,3.8,736339,Adelaida Buscato
205438,tm1059008,comedy,3.8,399499,Luz Stella Luengas
205439,tm1059008,comedy,3.8,373198,Inés Prieto


由于可能存在演员同名的情况，因此将 person_id 也保留了下来，以此来精准定位每一个演员。

In [21]:
actor_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205441 entries, 0 to 205440
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          205441 non-null  object 
 1   genres      205441 non-null  object 
 2   imdb_score  205441 non-null  float64
 3   person_id   205441 non-null  int64  
 4   name        205441 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 7.8+ MB


In [22]:
actor_data['person_id'] = actor_data['person_id'].astype(str)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9444\3634192818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actor_data['person_id'] = actor_data['person_id'].astype(str)


person_id的数据类型应为字符串（str），而非整数(int)

In [23]:
mean_score = actor_data.groupby(['genres','person_id'])['imdb_score'].mean()
mean_score

genres   person_id
action   1000         6.866667
         100007       7.000000
         100013       6.400000
         100019       6.500000
         100020       6.500000
                        ...   
western  993735       6.500000
         998673       7.300000
         998674       7.300000
         998675       7.300000
         99940        4.000000
Name: imdb_score, Length: 168881, dtype: float64

根据代码筛选出了各个类型电影的平均得分

In [24]:
reset_score = mean_score.reset_index()
reset_score

,genres,person_id,imdb_score
0,action,1000,6.866667
1,action,100007,7.000000
2,action,100013,6.400000
3,action,100019,6.500000
4,action,100020,6.500000
...,...,...,...
168876,western,993735,6.500000
168877,western,998673,7.300000
168878,western,998674,7.300000
168879,western,998675,7.300000


In [25]:
max_score = reset_score.groupby('genres')['imdb_score'].max()
max_score

genres
action           9.3
animation        9.3
comedy           9.2
crime            9.5
documentation    9.1
drama            9.5
european         8.9
family           9.3
fantasy          9.3
history          9.1
horror           9.0
music            8.8
reality          8.9
romance          9.2
scifi            9.3
sport            9.1
thriller         9.5
war              8.8
western          8.9
Name: imdb_score, dtype: float64

找出各类型影视的最高评分数据。

In [26]:
max_score_actor = pd.merge(max_score,reset_score,on =['genres','imdb_score'],how ='inner')
max_score_actor

,genres,imdb_score,person_id
0,action,9.3,12790
1,action,9.3,1303
2,action,9.3,21033
3,action,9.3,336830
4,action,9.3,86591
...,...,...,...
131,war,8.8,826547
132,western,8.9,22311
133,western,8.9,28166
134,western,8.9,28180


获得了各类型影视获得平均分最高的演员的person_id

In [27]:
name = actor_data[['person_id','name']].drop_duplicates()
name['person_id'].value_counts()

person_id
3748       1
1107852    1
23245      1
12678      1
1504837    1
          ..
17196      1
17200      1
17197      1
17199      1
378132     1
Name: count, Length: 49736, dtype: int64

In [28]:
max_name = pd.merge(max_score_actor,name,on ='person_id')
max_name

,genres,imdb_score,person_id,name
0,action,9.3,12790,Olivia Hack
1,scifi,9.3,12790,Olivia Hack
2,action,9.3,1303,Jessie Flower
3,animation,9.3,1303,Jessie Flower
4,family,9.3,1303,Jessie Flower
...,...,...,...,...
131,war,8.8,826547,Yuto Uemura
132,western,8.9,22311,Koichi Yamadera
133,western,8.9,28166,Megumi Hayashibara
134,western,8.9,28180,Unsho Ishizuka


获得了各类型影视获得平均分最高的演员的名字，但是顺序乱了。

In [29]:
max_name = max_name.sort_values('genres').reset_index().drop('index',axis=1)
max_name

,genres,imdb_score,person_id,name
0,action,9.3,12790,Olivia Hack
1,action,9.3,336830,André Sogliuzzo
2,action,9.3,21033,Zach Tyler
3,action,9.3,86591,Cricket Leigh
4,action,9.3,1303,Jessie Flower
...,...,...,...,...
131,war,8.8,826547,Yuto Uemura
132,western,8.9,28166,Megumi Hayashibara
133,western,8.9,28180,Unsho Ishizuka
134,western,8.9,22311,Koichi Yamadera


完成项目目标，获取了各类型影视获得平均分最高的演员的名字。

In [30]:
max_name.to_csv('Netflix_max_imdb_score_actor_name.csv')